In [1]:
from pyspark.sql import SparkSession
# Creating spark session
spark = SparkSession.builder.appName('Exercise').getOrCreate()

In [5]:
df = spark.read.csv('walmart_stock.csv',header=True,inferSchema=True)

In [6]:
df.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

In [11]:
# Printing list of column names
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [12]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [17]:
for i in df.head(5):
    print(i)
    print('\n')

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)


Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)


Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)


Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)


Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)




In [18]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

In [30]:
# Using number format function and .cast function to convert the data type
from pyspark.sql.functions import format_number
result = df.describe()
result.select(result['Summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              format_number(result['Volume'].cast('float'),2).alias('Volume'),
              format_number(result['Adj Close'].cast('float'),2).alias('Adj Close')).show()

+-------+--------+--------+--------+--------+-------------+---------+
|Summary|    Open|    High|     Low|   Close|       Volume|Adj Close|
+-------+--------+--------+--------+--------+-------------+---------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00| 1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|    67.24|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|     6.72|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|    50.36|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|    84.91|
+-------+--------+--------+--------+--------+-------------+---------+



In [37]:
df2 = df.withColumn('HV Ratio',df['High']/df['Volume'])
df2.select(['HV Ratio']).show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [53]:
df.createOrReplaceTempView('Data')
spark.sql( 'SELECT Date from Data where High = (SELECT MAX(High) from Data)').show()

+----------+
|      Date|
+----------+
|2015-01-13|
+----------+



In [60]:
# Importing mean function
from pyspark.sql.functions import mean,min,max
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [61]:
df.select(min('Volume'),max('Volume')).show()

+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|    2094900|   80898100|
+-----------+-----------+



In [62]:
# Using filter to filter the data
df.filter('Close < 60').count()

81

In [63]:
df.filter(df['Close']<60).count()

81

In [65]:
Days = df.filter('High > 80').count()
TotalDays = df.count()
Days/TotalDays*100

9.141494435612083

In [66]:
# Importing correlation function
from pyspark.sql.functions import corr
df.select (corr('High','Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



In [69]:
from pyspark.sql.functions import year
yeardf = df.withColumn('Year',year(df['Date']))

In [73]:
max_df = yeardf.groupBy('Year').max()
max_df.select('Year','max(High)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



In [80]:
from pyspark.sql.functions import month
monthdf = df.withColumn('Month',month(df['Date']))
monthdf.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+-----+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|Month|
+----------+------------------+------------------+------------------+------------------+--------+------------------+-----+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|    1|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|    1|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|    1|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|    1|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|    1|
|2012-01-10|    

In [85]:
monthdf.select('Month','Close').groupBy('Month').mean().orderBy('Month').show()

+-----+----------+-----------------+
|Month|avg(Month)|       avg(Close)|
+-----+----------+-----------------+
|    1|       1.0|71.44801958415842|
|    2|       2.0|  71.306804443299|
|    3|       3.0|71.77794377570092|
|    4|       4.0|72.97361900952382|
|    5|       5.0|72.30971688679247|
|    6|       6.0| 72.4953774245283|
|    7|       7.0|74.43971943925233|
|    8|       8.0|73.02981855454546|
|    9|       9.0|72.18411785294116|
|   10|      10.0|71.57854545454543|
|   11|      11.0| 72.1110893069307|
|   12|      12.0|72.84792478301885|
+-----+----------+-----------------+

